In [11]:
import numpy as np
import random

class Processo(object):
    def __init__(self, pnome, pio, ptam, prioridade, tempoChegada):
        self.nome = pnome
        self.io = pio  # Probability of performing I/O, initially zero
        self.tam = ptam  # Total timeslices needed to finish
        self.prio = prioridade  # Priority, not used now
        self.chegada = tempoChegada  # Arrival time
        self.remaining_time = ptam  # Remaining execution time
        self.start_time = None  # Time when the process first gets CPU
        self.finish_time = None  # Time when the process finishes execution

    def roda(self, tempo, quantum=None):  # Receive 'tempo' as a parameter
        if self.start_time is None:
            self.start_time = tempo  # Record the first time the process gets CPU

        if random.randint(1, 100) < self.io:  # Check if it performs I/O
            self.remaining_time -= 1
            print(self.nome, "performed I/O, remaining time", self.remaining_time)
            return 1, True  # True indicates it performed I/O

        if quantum is None or self.remaining_time < quantum:
            quantum = self.remaining_time
        self.remaining_time -= quantum

        if self.remaining_time == 0:
            self.finish_time = tempo + quantum  # Record the finish time

        print(self.nome, "ran for", quantum, "timeslice(s), remaining time", self.remaining_time)
        return quantum, False  # False indicates it did not perform I/O

class escalonador(object):  # Simple FCFS scheduler
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()  # Processes that arrive at time zero

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if self.prontos:
            return self.prontos.pop(0)  # FCFS: pick the first process
        else:
            return None
        


In [12]:
nprocs = 4
nomes = ['A', 'B', 'C', 'D']
chanceio = [0, 0, 0, 0]  # 0% chance of I/O
tamanho = np.array([3, 3, 3, 3])
total = tamanho.sum()
procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i], chanceio[i], tamanho[i], 0, 0))  # All processes arrive at time 0

In [13]:
# Simulation parameters
quantum = 2
tempoBloq = 1
escalonador = escalonador(procs)  # Instantiate the scheduler
bloqueados = []
tempo = 0  # Initialize 'tempo' before it's used
random.seed(0)

# Simulation loop
while total > 0:
    p = escalonador.proximo()
    if p is not None:
        # Update 'tempo' before the process runs
        current_tempo = tempo

        if p.remaining_time < quantum:
            actual_quantum = p.remaining_time
        else:
            actual_quantum = quantum

        # Process runs
        rodou, _ = p.roda(current_tempo, quantum=quantum)
        tempo += rodou  # Update 'tempo' after the process runs
        total -= rodou

        if p.remaining_time > 0:
            escalonador.pronto(p)
    else:
        # No process is ready; advance time
        tempo += 1

# Calculate average execution and response times
execution_times = []
response_times = []

for proc in procs:
    if proc.finish_time is not None and proc.start_time is not None:
        execution_time = proc.finish_time - proc.chegada
        response_time = proc.start_time - proc.chegada
        execution_times.append(execution_time)
        response_times.append(response_time)
        print(f"Process {proc.nome}: Execution Time = {execution_time}, Response Time = {response_time}")
    else:
        print(f"Process {proc.nome}: Did not finish execution.")

average_execution_time = sum(execution_times) / len(execution_times)
average_response_time = sum(response_times) / len(response_times)

print(f"\nAverage Execution Time: {average_execution_time}")
print(f"Average Response Time: {average_response_time}")

A ran for 2 timeslice(s), remaining time 1
B ran for 2 timeslice(s), remaining time 1
C ran for 2 timeslice(s), remaining time 1
D ran for 2 timeslice(s), remaining time 1
A ran for 1 timeslice(s), remaining time 0
B ran for 1 timeslice(s), remaining time 0
C ran for 1 timeslice(s), remaining time 0
D ran for 1 timeslice(s), remaining time 0
Process A: Execution Time = 9, Response Time = 0
Process B: Execution Time = 10, Response Time = 2
Process C: Execution Time = 11, Response Time = 4
Process D: Execution Time = 12, Response Time = 6

Average Execution Time: 10.5
Average Response Time: 3.0
